In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-21 14:31:16,446 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [64]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model')
model.init_sims(replace=True)

2017-06-21 14:40:15,925 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-21 14:40:23,184 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.* with mmap=None
2017-06-21 14:40:23,185 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.syn0.npy with mmap=None
2017-06-21 14:40:27,133 : INFO : setting ignored attribute syn0norm to None
2017-06-21 14:40:27,134 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.syn1neg.npy with mmap=None
2017-06-21 14:40:31,025 : INFO : setting ignored attribute cum_table to None
2017-06-21 14:40:31,026 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-21 14:40:34,508 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[(u'rs', 0.8790924549102783), (u'kk', 0.8604964017868042), (u'hahaha', 0.8581379652023315), (u'hahah', 0.8580098748207092), (u'hehe', 0.8567013740539551), (u'rsrs', 0.8455395698547363), (u'hehehe', 0.8401827812194824), (u'kkk', 0.8391278386116028), (u'hahahaha', 0.8145782947540283), (u'kkkk', 0.8133679032325745)]


In [4]:
print(model.most_similar(positive=[':D', ':/']))

[(u':(', 0.8271794319152832), (u';)', 0.8041931390762329), (u':)', 0.8021442890167236), (u':))', 0.7625139951705933), (u':c', 0.7616134881973267), (u'*-*', 0.7583409547805786), (u'=)', 0.7568512558937073), (u':3', 0.7542413473129272), (u':P', 0.7475375533103943), (u'^^', 0.744448184967041)]


In [5]:
print(model.most_similar(positive=['fofo', 'cute']))

[(u'fofinho', 0.7537943124771118), (u'lindinho', 0.7081056237220764), (u'meigo', 0.7032421827316284), (u'fofooo', 0.6875720620155334), (u'fofoooo', 0.6844723224639893), (u'fofo!', 0.6831097602844238), (u'cute!', 0.6810519695281982), (u'mordivel', 0.6774948835372925), (u'mord\xedvel', 0.6755068302154541), (u'fofo,', 0.6729605197906494)]


In [6]:
num_features= 300

In [7]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [8]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [65]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [66]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            if word in stopwords.words('portuguese'):
                continue
            tokens.append(word)
    return tokens

In [67]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [68]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-21 14:45:29,870 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:30,118 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:30,124 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:30,619 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:30,814 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:31,111 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:31,675 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:31,749 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:31,927 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:32,057 : WARNING : cannot compute similarity with no input []
2017-06-21 14:45:32,109 : WARNING : cannot compute similarity with no input []


CPU times: user 3.05 s, sys: 8.6 ms, total: 3.06 s
Wall time: 3.05 s


In [ ]:
del model

In [69]:
X_train_word_average.shape

(727, 300)

In [70]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

## Extraversion evaluation

In [71]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [72]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   25.7s finished


CPU times: user 7.19 s, sys: 657 ms, total: 7.84 s
Wall time: 33.8 s


In [73]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [74]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,101,41
1,81,89


In [75]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.55      0.71      0.62       142
          1       0.68      0.52      0.59       170

avg / total       0.63      0.61      0.61       312



In [76]:
accuracy_score(yt, yp)

0.60897435897435892

In [77]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [81]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [82]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       387
          1       1.00      1.00      1.00       340

avg / total       1.00      1.00      1.00       727



In [83]:
print accuracy_score(ytt, ytp)

0.998624484182


## Agreableness evaluation

In [84]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['agreeabeness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   25.4s finished


CPU times: user 4.94 s, sys: 94.4 ms, total: 5.03 s
Wall time: 30.4 s


In [85]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['agreeabeness_m']

In [86]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,67,74
1,72,99


In [87]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.48      0.48      0.48       141
          1       0.57      0.58      0.58       171

avg / total       0.53      0.53      0.53       312



In [88]:
accuracy_score(yt, yp)

0.53205128205128205

In [31]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [32]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['agreeabeness_m']

In [33]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       356
          1       1.00      1.00      1.00       371

avg / total       1.00      1.00      1.00       727



In [34]:
print accuracy_score(ytt, ytp)

1.0


## Conscientiousness evaluation

In [35]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['conscientiousness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.5s finished


CPU times: user 4.81 s, sys: 94.4 ms, total: 4.91 s
Wall time: 31.4 s


In [36]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['conscientiousness_m']

In [37]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,91,76
1,51,94


In [38]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.64      0.54      0.59       167
          1       0.55      0.65      0.60       145

avg / total       0.60      0.59      0.59       312



In [39]:
accuracy_score(yt, yp)

0.59294871794871795

In [40]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [41]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['conscientiousness_m']

In [42]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       356
          1       0.99      1.00      1.00       371

avg / total       1.00      1.00      1.00       727



In [43]:
print accuracy_score(ytt, ytp)

0.997248968363


## Neuroticism evaluation

In [44]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['neuroticism_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.2s finished


CPU times: user 5.03 s, sys: 71.8 ms, total: 5.1 s
Wall time: 29.3 s


In [45]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['neuroticism_m']

In [46]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,99,64
1,82,67


In [47]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.55      0.61      0.58       163
          1       0.51      0.45      0.48       149

avg / total       0.53      0.53      0.53       312



In [48]:
accuracy_score(yt, yp)

0.53205128205128205

In [49]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [50]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['neuroticism_m']

In [51]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       372
          1       0.99      1.00      1.00       355

avg / total       1.00      1.00      1.00       727



In [52]:
print accuracy_score(ytt, ytp)

0.997248968363


## Openess evaluation

In [53]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['openness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.1s finished


CPU times: user 4.85 s, sys: 82.4 ms, total: 4.93 s
Wall time: 29 s


In [54]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['openness_m']

In [55]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,76,60
1,79,97


In [56]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.49      0.56      0.52       136
          1       0.62      0.55      0.58       176

avg / total       0.56      0.55      0.56       312



In [57]:
accuracy_score(yt, yp)

0.55448717948717952

In [58]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [59]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['openness_m']

In [60]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       360
          1       0.99      1.00      1.00       367

avg / total       1.00      1.00      1.00       727



In [61]:
print accuracy_score(ytt, ytp)

0.995873452545
